# PLAN

**Strategy 1:** Get more frames from already tracked videos

**Strategy 2:** Get random frames from all videos

1. Get all marked videos - paths to .avi and .csv
2. Threshold p-values (and maybe edge lenghts)
3. Extract frames manually from .avi
4. Save frames to the dlc project
5. Re-train

In [58]:
import numpy as np
import h5py
import deeplabcut
import pandas as pd
import matplotlib.pyplot as plt

# Append base directory
import os,sys,inspect
thispath = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parpath = os.path.dirname(thispath)
sys.path.append(parpath)
print("Appended script directory", parpath)

%load_ext autoreload
%autoreload 2

from lib.os_lib import getfiles_walk, get_files
from lib.qt_wrapper import gui_fname, gui_fnames, gui_fpath
from lib.parse_dlc_results import dlc_csv_composite_crawl, dlc_fix_old_h5
from lib.video_convert_lib import save_frames_cv2

Appended script directory /home/sipila/work/dlc-scripts/scripts
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
project_config_path = gui_fname("Get project config file", "./", "*.yaml")

In [3]:
root_crawl_path = gui_fpath("Select path for crawler", "./")
output_path = gui_fpath("Select output path", root_crawl_path)

In [4]:
# 1. Load all h5 files
aviList = getfiles_walk(root_crawl_path, [".avi"])
vidPaths = [os.path.join(path, name) for path, name in aviList]

In [91]:
deeplabcut.extract_outlier_frames(project_config_path, [vidPaths[5]], outlieralgorithm='uncertain',epsilon=100 , p_bound=0.2, automatic=True)

5it [00:00, 44.67it/s]

Method  uncertain  found  228  putative outlier frames.
Do you want to proceed with extracting  100  of those?
If this list is very large, perhaps consider changing the paramters (start, stop, p_bound, comparisonbodyparts) or use a different method.
Frames from video 2018.10.22_09_05_45  already extracted (more will be added)!
Loading video...
Duration of video [s]:  29.12 , recorded @  200.0 fps!
Overall # of frames:  5824 with (cropped) frame dimensions: 
Kmeans-quantization based extracting of frames from 0.0  seconds to 29.12  seconds.
Extracting and downsampling... 228  frames from the video.


228it [00:04, 45.95it/s]


Kmeans clustering ... (this might take a while)
Let's select frames indices: [3510, 2918, 4770, 2587, 30, 4801, 4556, 4749, 2102, 431, 2943, 3070, 4544, 4739, 2888, 3422, 4787, 2842, 4755, 4571, 2572, 4760, 2986, 4536, 4781, 2930, 2936, 63, 2595, 2105, 2098, 4797, 4567, 4585, 2901, 2835, 4778, 4807, 4547, 4539, 3515, 4789, 3547, 4751, 4765, 4683, 4783, 4732, 2941, 4540, 2946, 4794, 2097, 2922, 4774, 2935, 4758, 3075, 4551, 2914, 2110, 4573, 4535, 36, 2928, 2987, 2112, 4776, 4537, 4565, 4804, 4785, 2578, 4759, 4562, 2584, 4766, 3425, 2592, 4730, 2870, 3507, 3511, 4763, 2834, 4550, 2101, 2939, 4784, 4799, 433, 3505, 2933, 4791, 4541, 2590, 4782, 4734, 4772, 3420]
New video was added to the project! Use the function 'extract_frames' to select frames for labeling.
The outlier frames are extracted. They are stored in the subdirectory labeled-data\2018.10.22_09_05_45.
Once you extracted frames for all videos, use 'refine_labels' to manually correct the labels.


In [87]:
deeplabcut.extract_outlier_frames?

# TODO-PIA

1. Play with 1-2 videos, figure out good outlier selection mechanism
2. Goto Config file, change number of frames to larger number (e.g 100)
3. Figure out which videos to use for outliers (maybe based on CSV file)


In [ ]:
# 2. Extract traces and video names
# 3. Load DLC project folder
# 4. Filter bad p-values
# 5. Extract bad frames, save as images to DLC project folder

In [ ]:
q = 0.16002344069833624

rezTable = []

for vidPath in vidPaths:
    if "norun" not in vidPath:
        vidKey = os.path.splitext(os.path.basename(vidPath))[0]
        dlcKey = "DeepCut_resnet50"
        
        # 1. Get CSV filename from video name
        csvNames = get_files(os.path.dirname(vidPath), '.csv')
        csvNames = [csv for csv in csvNames if (vidKey in csv) and (dlcKey in csv)]
        
        if len(csvNames) != 1:
            print("Skipping", csvNames, vidKey)
        else:
            # 2. Load CSV file, get histogram of p-values for wrist
            csvPathName = os.path.join(os.path.dirname(vidPath), csvNames[0])
            df = pd.read_csv(csvPathName)
            
            # Renaming columns of the dataset
            bodyparts = list(df.iloc[0])
            valueKeys = list(df.iloc[1])
            newCols = [b + ',' + v for b,v in zip(bodyparts, valueKeys)]
            df.columns = newCols
            
            wristLikelihoods = np.array(list(df['Wrist,likelihood'][2:])).astype(float)
            
            avgWristPValPower = np.mean(np.log(wristLikelihoods))
            
            rezTable += [[vidKey, avgWristPValPower]]
            
            # 3. If hist is shitty, pick this video: 
            if np.abs(avgWristPValPower) > q:
                deeplabcut.extract_outlier_frames(project_config_path, [vidPath], outlieralgorithm='uncertain',epsilon=100 , p_bound=0.2, automatic=True)

0it [00:00, ?it/s]

Method  uncertain  found  365  putative outlier frames.
Do you want to proceed with extracting  100  of those?
If this list is very large, perhaps consider changing the paramters (start, stop, p_bound, comparisonbodyparts) or use a different method.
Loading video...
Duration of video [s]:  29.22 , recorded @  200.0 fps!
Overall # of frames:  5844 with (cropped) frame dimensions: 
Kmeans-quantization based extracting of frames from 0.0  seconds to 29.22  seconds.
Extracting and downsampling... 365  frames from the video.


365it [00:07, 46.31it/s]


Kmeans clustering ... (this might take a while)
Let's select frames indices: [1886, 5017, 4295, 4571, 5237, 4130, 1931, 3587, 1412, 3662, 3720, 4093, 1941, 4991, 5078, 5284, 4339, 4547, 1401, 4330, 4319, 4564, 4444, 3624, 4526, 4055, 1928, 5229, 4134, 1889, 4350, 4587, 1418, 4998, 5007, 5059, 4111, 5117, 3658, 5052, 1881, 1407, 1935, 4600, 4281, 4323, 4336, 3745, 4993, 4582, 4530, 1698, 3718, 5100, 4440, 5040, 4352, 5291, 1917, 4307, 1933, 4335, 1944, 4327, 4559, 4538, 4343, 4557, 4137, 4535, 3726, 1937, 4567, 4090, 3646, 4442, 5026, 4347, 4348, 1940, 1884, 4283, 3663, 1887, 1410, 4097, 4543, 4987, 3653, 1921, 1396, 1414, 4313, 1403, 4278, 4554, 4578, 1882, 3710, 5032]
Creating the symbolic link of the video
New video was added to the project! Use the function 'extract_frames' to select frames for labeling.
The outlier frames are extracted. They are stored in the subdirectory labeled-data\2018.10.22_09_06_22.
Once you extracted frames for all videos, use 'refine_labels' to manually cor

0it [00:00, ?it/s]

...

1043it [00:29, 35.39it/s]


Kmeans clustering ... (this might take a while)
Let's select frames indices: [4906, 170, 2827, 302, 2648, 2475, 1809, 1628, 2744, 227, 316, 2880, 2589, 2681, 248, 2573, 214, 2878, 4691, 5033, 1268, 1801, 273, 573, 2542, 267, 296, 2612, 4482, 1439, 1623, 237, 1517, 1003, 1806, 2574, 2724, 2631, 225, 1631, 2586, 4554, 2571, 992, 2666, 1812, 2791, 2695, 277, 2502, 2548, 2886, 220, 324, 4798, 137, 4758, 2872, 2498, 2577, 2685, 232, 318, 2671, 2765, 2854, 236, 16, 2523, 149, 4967, 2608, 209, 2656, 4699, 2564, 1356, 2487, 2835, 1805, 2775, 2635, 2630, 284, 204, 33, 2808, 5059, 241, 288, 4874, 2505, 2708, 2849, 2490, 206, 250]
Creating the symbolic link of the video
New video was added to the project! Use the function 'extract_frames' to select frames for labeling.
The outlier frames are extracted. They are stored in the subdirectory labeled-data\2005.01.01_20_44_53.
Once you extracted frames for all videos, use 'refine_labels' to manually correct the labels.
Method  uncertain  found  1633  p

1it [00:00,  8.04it/s]

Duration of video [s]:  27.005 , recorded @  200.0 fps!
Overall # of frames:  5401 with (cropped) frame dimensions: 
Kmeans-quantization based extracting of frames from 0.0  seconds to 27.0  seconds.
Extracting and downsampling... 1633  frames from the video.


656it [00:19, 33.15it/s]

In [95]:
rezDF = pd.DataFrame(rezTable, columns=["VidName", "PValPow"])
absPow = np.abs(rezDF["PValPow"])

nVidWant = 350
nVidHave = len(absPow)
q = np.quantile(absPow, 1 - nVidWant / nVidHave)

plt.figure(figsize=(10,10))
plt.hist(absPow, bins='auto')
plt.axvline(x = q, color='r')
plt.xscale('log')
plt.yscale('log')
plt.show()

print(q)

0.16002344069833624
